In [1]:
%load_ext autoreload
%autoreload 2
import sys
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
from os.path import join as oj
import pandas as pd
import pickle as pkl
import models
import util
import os
import config
from config import ATTR_TO_INDEX
import viz
import scipy.stats
from tqdm import tqdm
import figs
import matplotlib.image as mpimg
import sklearn.decomposition
import sklearn.manifold
import seaborn as sns
import data
import transects
import face_recognition
import sklearn.metrics
from matching import *
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox


df = data.load_all_labs()
df = df.set_index('fname_id')

DIR_ORIG = '../data/celeba-hq/ims/'
reg = 0.1
DIR_GEN = oj(f'../data_processed/celeba-hq/generated_images_{reg}')

# get fnames
fname_nps = [f for f in sorted(os.listdir(DIR_GEN)) if 'npy' in f] # these start at 00001
fname_ids = np.array([f[:-4] for f in fname_nps])
idxs_calculated = np.array([int(x) - 1 for x in fname_ids]) # this starts at 0

# trim df to only have the relevant ids
df = df.loc[fname_ids]

# load the linear model in latent space
coefs, intercepts = transects.get_directions()
coefs = np.array(coefs).squeeze()
intercepts = np.array(intercepts)


# load latents and calculate dists
print('loading latents...')
latents = np.array([np.load(oj(DIR_GEN, f)) for f in fname_nps])
lats = get_lat(latents)
preds = latents.mean(axis=1) @ coefs.T + intercepts.T
weights = np.zeros(preds.shape[1])
# print(ATTR_TO_INDEX)
# weights[ATTR_TO_INDEX['skin-color']] = 1e2
vecs = join_vecs(preds, lats, weights)

print('calculating dists...')
dists_gan = get_dists(vecs)
print('done!')

# load pairwise facial dicts
print('loading facial rec dists...')
dists_facial = np.load(open('processed/13_facial_dists_pairwise.npy', 'rb')) # pkl.load()
dists_facial = dists_facial[idxs_calculated, :][:, idxs_calculated]
print('done!')

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/python3/lib/pyt

loading cached labels


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.ridge module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklear

loading latents...
calculating dists...
done!
loading facial rec dists...
done!


# do matching

### find matching for an im

In [ ]:
# img 38 is a good gender example
for im_idx in range(10):# range(38, 45):
    
    # select subset of indices to use for matching
    idxs = np.ones(df.shape[0]).astype(bool)
#     idxs = (df['gender'] > 0).values
#     idxs = (df['race'] == 'Black').values
#     idxs = (df['Eyeglasses'] > 0).values
    dists_im = dists_gan[im_idx][idxs] # first select row, then select vals
#     dists_im = dists_facial[im_idx][idxs]
    fname_ids_for_matching = fname_ids[idxs]    
    
    closest_match_vals, closest_matches_fnames = calc_matches(dists_im, fname_ids_for_matching)
    # print(closest_matches_fnames)
    
    # load images
    N_MATCHES_TO_PLOT = 5
    fname_id = fname_ids[im_idx]
    im_orig = mpimg.imread(oj(DIR_ORIG, f'{fname_id}.jpg'))
    im_rec = mpimg.imread(oj(DIR_GEN, f'{fname_id}.png'))
    im_matches = [mpimg.imread(oj(DIR_GEN, f'{fname_match}.png'))
                  for fname_match in closest_matches_fnames[:N_MATCHES_TO_PLOT]]
    
    # plt images
    util.plot_row([im_orig, im_rec] + im_matches,
                  annot_list=['orig', 'rec'] + closest_match_vals[:N_MATCHES_TO_PLOT].round(3).tolist(), dpi=150)
    plt.show()
    # print(closest_matches, closest_matches_fnames)
# show_matches(dists_gan, DIR_ORIG, DIR_GEN, im_nums=range(5, 10))

**simple dim reduction**

In [ ]:
def plot_image(xs, ys, im):
    '''Note: should normalize x/y coords to 0-1 before plotting
    '''
    for idx, (x, y) in tqdm(enumerate(zip(xs, ys))):
        bb = Bbox.from_bounds(x, y, IM_SIZE, IM_SIZE)  
        bb2 = TransformedBbox(bb, ax.transData)
        bbox_image = BboxImage(bb2, origin=None, clip_on=False)
        bbox_image.set_data(im[idx])
#          bbox_image.set_alpha(1.0)
        ax.add_artist(bbox_image)
    return ax

# plot
N_IMS = 1000
IM_SIZE = 0.025
N_PLOT = 1000

fig = plt.figure(figsize=(20, 20), dpi=100)
ax = fig.add_subplot(111)
ims = [mpimg.imread(oj(DIR_ORIG, f))[::2, ::2] for f in df.fname_final[:N_IMS]]
l = sklearn.decomposition.PCA(n_components=2).fit_transform(lats[:N_IMS])
# l = sklearn.manifold.TSNE().fit_transform(lats[:N_IMS])
l = (l - l.min(axis=0)) / (l.max(axis=0) - l.min(axis=0))
ax = plot_image(l[:N_PLOT, 0], l[:N_PLOT, 1], ims)
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
# plt.xlim((0.5, 1))
# plt.ylim((0, 0.5))
plt.show()

# plots for specific ids/matches

**make some plots for a specific match**

In [ ]:
im_idx = 38 # img 38 is a good gender example
fname_id2 = '02638' # this is the match
# if all images are calculated, then this is just 5-char string of im_idx + 1
fname_id = fname_ids[im_idx] 


# idxs
dists_im = dists_gan[im_idx]

im = mpimg.imread(oj(DIR_ORIG, f'{fname_id}.jpg'))
im2 = mpimg.imread(oj(DIR_ORIG, f'{fname_id2}.jpg'))

d1 = df[df.id == df.loc[fname_id].id]
d2 = df[df.id == df.loc[fname_id2].id]



util.plot_row([im, im2])
util.plot_row([mpimg.imread(oj(DIR_ORIG, f)) for f in d1.fname_final])
util.plot_row([mpimg.imread(oj(DIR_ORIG, f)) for f in d2.fname_final])

**look at images / reconstructions of the same person**

In [ ]:
d = df[df['count_with_this_id'] >= df['count_with_this_id'].max() - 1]
dd = d[d.id == d.iloc[0].id]
util.plot_row([mpimg.imread(oj(DIR_ORIG, f)) for f in dd.fname_final][:5])
util.plot_row([mpimg.imread(oj(DIR_GEN, f'{index}.png')) for index in dd.index][:5])

# evaluate matching with metrics

**how often do we return the same id?**

In [2]:
dists_match_names = ['facial', 'gan', 'gan_constrained']

In [3]:
d = df[df['count_with_this_id'] > 1]
# print(d.shape)
accs_keys = ['acc_top1', 'acc_top5', 'acc_top10']
attr_keys = [kk for kk in df.keys() if not 'scores' in kk]
r = {
    k: [] for k in accs_keys + attr_keys
}

for dists_match_name in dists_match_names:
    if dists_match_name == 'facial':
        dists_match = dists_facial
    elif dists_match_name == 'gan':
        dists_match = dists_gan
    elif dists_match_name == 'gan_constrained':
        dists_match = dists_gan + (dists_facial > 0.6) * 1e3 # constraint for missclassificaiton
    lists = {
        k: [] for k in r.keys()
    }
    
    for im_idx in tqdm(range(1000)):
        id_orig = df.iloc[im_idx].id

        # id retention
        dists_im = dists_match[im_idx]
        matched_idxs = np.argsort(dists_im)
        matched_ids = df.iloc[matched_idxs].id.values # note - this needs to be df not d to get the proper indices from dists
        lists['acc_top1'].append(id_orig in matched_ids[:1])
        lists['acc_top5'].append(id_orig in matched_ids[:5])
        lists['acc_top10'].append(id_orig in matched_ids[:10])
        
        # are other feats matched?
        d = df.iloc[matched_idxs[:10]]
        for k in attr_keys:
            orig = df.iloc[im_idx]
            # print(d[k], orig[k], len(lists[k]))
            lists[k].append(np.mean(d[k] == orig[k]))
        
    for k in lists.keys():
        r[k].append(np.mean(lists[k]))
#     r['acc_top1'].append(np.mean(acc_top1))
#     r['acc_top5'].append(np.mean(acc_top5))
#     r['acc_top10'].append(np.mean(acc_top10))
r = pd.DataFrame.from_dict(r)
r.to_pickle('processed/13_dist_stats.pkl')

100%|██████████| 1000/1000 [01:14<00:00, 13.45it/s]


In [3]:
import pandas as pd
r = pd.read_pickle('processed/13_dist_stats.pkl').round(3)
r.index = dists_match_names
for k in r.keys():
    if r.loc['gan', k] > r.loc['facial', k]:
        print(k, f"{r.loc['gan', k] - r.loc['facial', k]:0.3f}")
r.style.background_gradient()

skin-color 0.006
Bangs 0.029
Blurry 0.002
Eyeglasses 0.006
Pale_Skin 0.006
Wavy_Hair 0.022
Wearing_Hat 0.021


,acc_top1,acc_top5,acc_top10,idx,orig_idx,orig_file,proc_md5,final_md5,id,fname_final,count_with_this_id,gender,hair-length,facial-hair,makeup,skin-color,age,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,face_name_align_pred,race_pred,race4_pred,gender_pred,age_pred,img_names_pred
facial,0.906000,0.935000,0.937000,0.000000,0.000000,0.000000,0.000000,0.000000,0.596000,0.000000,0.622000,0.989000,0.866000,0.924000,0.725000,0.923000,0.871000,0.918000,0.683000,0.718000,0.683000,0.987000,0.791000,0.746000,0.744000,0.815000,0.849000,0.985000,0.741000,0.856000,0.926000,0.932000,0.960000,0.952000,0.972000,0.725000,0.662000,0.973000,0.614000,0.956000,0.827000,0.924000,0.800000,0.923000,0.672000,0.867000,0.804000,0.940000,0.637000,0.732000,0.655000,0.668000,0.956000,0.841000,0.705000,0.930000,0.871000,0.000000,0.881000,0.935000,0.942000,0.725000,0.000000
gan,0.178000,0.250000,0.291000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046000,0.000000,0.095000,0.843000,0.848000,0.840000,0.683000,0.929000,0.749000,0.812000,0.584000,0.617000,0.647000,0.983000,0.820000,0.540000,0.615000,0.694000,0.790000,0.987000,0.676000,0.730000,0.920000,0.908000,0.966000,0.927000,0.956000,0.683000,0.609000,0.842000,0.573000,0.948000,0.807000,0.840000,0.741000,0.929000,0.579000,0.850000,0.776000,0.898000,0.609000,0.658000,0.677000,0.622000,0.977000,0.769000,0.700000,0.868000,0.749000,0.000000,0.632000,0.776000,0.842000,0.693000,0.000000
gan_constrained,0.630000,0.815000,0.867000,0.000000,0.000000,0.000000,0.000000,0.000000,0.410000,0.000000,0.444000,0.988000,0.869000,0.922000,0.720000,0.922000,0.860000,0.912000,0.668000,0.707000,0.685000,0.984000,0.794000,0.688000,0.727000,0.798000,0.845000,0.986000,0.715000,0.844000,0.925000,0.929000,0.966000,0.950000,0.969000,0.720000,0.652000,0.974000,0.604000,0.956000,0.823000,0.922000,0.769000,0.922000,0.654000,0.868000,0.790000,0.939000,0.630000,0.722000,0.651000,0.672000,0.970000,0.848000,0.702000,0.924000,0.860000,0.000000,0.840000,0.920000,0.948000,0.732000,0.000000


In [5]:
rename = {
    'acc_top1': 'ID (top1)',
    'gender': 'Gender',
    'race_pred': 'Race'
}
r.index = ['Facial-rec dist', 'GAN dist', 'Combined']
r2 = r[['acc_top1', 'gender', 'race_pred', 'Mustache', 'Blurry', 'Eyeglasses', 'Bangs', 'Wearing_Hat']].rename(columns=rename) * 100

In [7]:
print(r2.to_latex())

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  ID (top1) &  Gender &  Race &  Mustache &  Blurry &  Eyeglasses &  Bangs &  Wearing\_Hat \\
\midrule
Facial-rec dist &       90.6 &    98.9 &  88.1 &      95.6 &    98.5 &        96.0 &   79.1 &         95.6 \\
GAN dist        &       17.8 &    84.3 &  63.2 &      94.8 &    98.7 &        96.6 &   82.0 &         97.7 \\
Combined        &       63.0 &    98.8 &  84.0 &      95.6 &    98.6 &        96.6 &   79.4 &         97.0 \\
\bottomrule
\end{tabular}

